#Rich Johnson, Charles Maalouf, Rajesh Thallam

[richard.johnson@ischool.berkeley.edu, cmaalouf@berkeley.edu, rajesh.thallam@ischool.berkeley.edu]

W261 - HW 5 Group

8 Oct 2015

##HW 5.0
What is a data warehouse? What is a Star schema? When is it used?



1. A data warehouse is a central location for bringing together multiple data sources for analysis or storage. 

2. Star Schema is a model for organizing data into *facts* and *dimensions*. Facts are measurable pieces of information (such as sales). Dimensions are reference information (Name, Date, Location). This schema gets its name from the shape that is formed when diagramming it.

3. Star Schema is used in data warehouses to query large data sets. Each dimension is associated to a fact table via a key. By combining keys, every piece of data can be uniquely identified.

##HW 5.1
In the database world What is 3NF? Does machine learning use data in 3NF? If so why? 
In what form does ML consume data?
Why would one use log files that are denormalized?


1. 3NF is short for third-normal form. It is a way of normalizing a database to reduce redundancy. In 3NF, none of the values in a table are determined by a non-prime key. e.g. If a table lists drs, their specialties, and where they went to med school, this would not be 3NF if there exists any doctors with multiple specialties, as the keys DR or Med School would not be prime.

2. Machine Learning does use 3NF in order to ensure data is in a clean form to simplify its usage. ML consumes data in a rigidly structured form; it requires features to be explicit. Using log files that are denormalized can allow for easier consumption through repeated data at the cost of more memory (or disk space) usage. 

##HW 5.2
Using MRJob, implement a hashside join (memory-backed map-side) for left, 
right and inner joins. Run your code on the  data used in HW 4.4: (Recall HW 4.4: Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.)
:

Justify which table you chose as the Left table in this hashside join.

Please report the number of rows resulting from:

(1) Left joining Table Left with Table Right
(2) Right joining Table Left with Table Right
(3) Inner joining Table Left with Table Right


In [18]:
%%writefile hw_5_2_mrjob.py
#!/usr/bin/python

from mrjob.job import MRJob
from mrjob.compat import get_jobconf_value

class CleanLeftTable(MRJob):
    def mapper(self, _, line):
        line = line.split(',')
        if line[0] == 'A':
            yield line[1], line[4]
        
class LeftJoin(MRJob):
    def __init__(self, args):
        super(LeftJoin, self).__init__(args)
        self.lt = {}
        self.rt = {}
    
    def mapper_init(self):
        lt_file = str(get_jobconf_value('left.table'))
        with open(lt_file, 'r') as fh:
            for l in fh.readlines():
                l = l.strip().split(',')
                self.lt[l[0]] = l[1]
                
    def mapper(self, _, line):
        line = line.strip().split(',')
        webpageid = line[1]
        visitorid = line[4]
        if webpageid not in self.rt.keys():
            self.rt[webpageid] = []
        self.rt[webpageid].append(visitorid)
        
    def mapper_final(self):
        for webpageid in self.lt.keys():
            try:
                for visitor in self.rt[webpageid]:
                    yield (self.lt[webpageid], webpageid), visitor
            except KeyError:
                yield (self.lt[webpageid], webpageid), None


class RightJoin(MRJob):
    def __init__(self, args):
        super(RightJoin, self).__init__(args)
        self.lt = {}
        self.rt = {}

    def mapper_init(self):
        lt_file = str(get_jobconf_value('left.table'))
        with open(lt_file, 'r') as fh:
            for l in fh.readlines():
                l = l.strip().split(',')
                self.lt[l[0]] = l[1]

    def mapper(self, _, line):
        line = line.strip().split(',')
        webpageid = line[1]
        visitorid = line[4]
        if webpageid not in self.rt.keys():
            self.rt[webpageid] = []
        self.rt[webpageid].append(visitorid)

    def mapper_final(self):
        for webpageid in self.rt.keys():
            for user in self.rt[webpageid]:
                try:
                    yield (self.lt[webpageid], webpageid), user
                except KeyError:
                    yield (None, webpageid), user

class InnerJoin(MRJob):
    def __init__(self, args):
        super(InnerJoin, self).__init__(args)
        self.lt = {}
        self.rt = {}

    def mapper_init(self):
        lt_file = str(get_jobconf_value('left.table'))
        with open(lt_file, 'r') as fh:
            for l in fh.readlines():
                l = l.strip().split(',')
                self.lt[l[0]] = l[1]

    def mapper(self, _, line):
        line = line.strip().split(',')
        webpageid = line[1]
        visitorid = line[4]
        if webpageid not in self.rt.keys():
            self.rt[webpageid] = []
        self.rt[webpageid].append(visitorid)

    def mapper_final(self):
        for webpageid in self.lt.keys():
            try:
                for visitor in self.rt[webpageid]:
                    yield (self.lt[webpageid], webpageid), visitor
            except KeyError:
                pass


Overwriting hw_5_2_mrjob.py


In [19]:
%load_ext autoreload
%autoreload 2

from hw_5_2_mrjob import CleanLeftTable, LeftJoin, RightJoin, InnerJoin

right_table = 'hw_4_2.out'
left_table = 'anonymous-msweb.data'
clean_lt = '/media/sf_Google_Drive/Berkeley/W261/Week 6/left_table.txt'

clean_left_table_mr = CleanLeftTable(args=[left_table])
with open(clean_lt, 'w') as fh:
    with clean_left_table_mr.make_runner() as runner:
        runner.run()
        for line in runner.stream_output():
            key, value = clean_left_table_mr.parse_output_line(line)
            fh.write('%s, %s\n' % (key, value))


left_join_mr = LeftJoin(args=[right_table, '--jobconf',
                              'left.table='+clean_lt])
with open('left_join.out', 'w') as fh:
    with left_join_mr.make_runner() as runner:
        runner.run()
        for line in runner.stream_output():
            key, value = left_join_mr.parse_output_line(line)
            #Needs text cleanup, but otherwise, good
            fh.write('%s, %s\n' % (key, value))

right_join_mr = RightJoin(args=[right_table, '--jobconf',
                              'left.table='+clean_lt])
with open('right_join.out', 'w') as fh:
    with right_join_mr.make_runner() as runner:
        runner.run()
        for line in runner.stream_output():
            key, value = right_join_mr.parse_output_line(line)
            #Needs text cleanup, but otherwise, good
            fh.write('%s, %s\n' % (key, value))

inner_join_mr = InnerJoin(args=[right_table, '--jobconf',
                              'left.table='+clean_lt])
with open('inner_join.out', 'w') as fh:
    with inner_join_mr.make_runner() as runner:
        runner.run()
        for line in runner.stream_output():
            key, value = inner_join_mr.parse_output_line(line)
            #Needs text cleanup, but otherwise, good
            fh.write('%s, %s\n' % (key, value))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
!wc -l left_join.out
!wc -l right_join.out
!wc -l inner_join.out

98663 left_join.out
98654 right_join.out
98654 inner_join.out


###Results
I chose to clean the 'anonymous-msweb.data' file and create a small table. Based on the async videos, I elected to use this table as my left table because it was the smaller of the two.

##HW 5.3 
For the remainder of this assignment you will work with a large subset 
of the Google n-grams dataset,

https://aws.amazon.com/datasets/google-books-ngrams/

which we have placed in a bucket on s3:

s3://filtered-5grams/

In particular, this bucket contains (~200) files in the format:

	(ngram) \t (count) \t (pages_count) \t (books_count)

Do some EDA on this dataset using mrjob, e.g., 

- Longest 5-gram (number of characters)
- Top 10 most frequent words (count), i.e., unigrams
- Most/Least densely appearing words (count/pages_count) sorted in decreasing order of relative frequency (Hint: save to PART-000* and take the head -n 1000)
- Distribution of 5-gram sizes (counts) sorted in decreasing order of relative frequency. (Hint: save to PART-000* and take the head -n 1000)
OPTIONAL Question:
- Plot the log-log plot of the frequency distributuion of unigrams. Does it follow power law distribution?

For more background see:
https://en.wikipedia.org/wiki/Log%E2%80%93log_plot
https://en.wikipedia.org/wiki/Power_law


In [ ]:
%%writefile hw_5_3_mrjob.py
#!/usr/bin/python

from mrjob.job import MRJob
import operator

class EDA(MRJob):
    def mapper(self, _, line):
        line = line.lower().strip().split('\t')

        # Longest 5-gram (number of characters)
        ngram_length = len(line[0])
        yield('*LENGTH', ngram_length)

        # Top 10 most frequent words
        # Most/Least densely appearing words
        ngram = line[0].split() #yield each one count times
        for word in ngram:
            yield word, (int(line[1]), int(line[2]))


    def reducer_init(self):
        self.word_counts = {}

    def reducer(self, key, value):
        if key == '*LENGTH':
            dist = {}
            for v in value:
                if v in dist.keys():
                    dist[v] += 1
                else:
                    dist[v] = 1
            for key in sorted(dist.keys(), reverse=True):
                yield 'LENGTH_DISTRIBUTION', (key, dist[key])

        else:
            self.word_counts[key] = [0, 0]
            counts = 0
            pages = 0
            for v in value:
                self.word_counts[key][0] += v[0]
                self.word_counts[key][1] += v[1]

            self.word_counts[key][1] = 1.0 * self.word_counts[key][0] / self.word_counts[key][1]

    def reducer_final(self):
        for w in sorted(self.word_counts.iteritems(), key=lambda (k,v): operator.itemgetter(0)(v),
                        reverse=True):
            yield ('COUNT', w[0]), w[1][0]
        for w in sorted(self.word_counts.iteritems(), key=lambda (k,v): operator.itemgetter(1)(v),
                        reverse=True):
            yield ('DENSITY', w[0]), w[1][1]

if __name__ == '__main__':
    EDA.run()

In [ ]:
from hw_5_3_mrjob import EDA

eda_mr = EDA(args=['-r', 'emr', 's3://filtered-5grams', '--conf-path', 'mrjob.conf',
                   '--no-output', '--output-dir', 's3://r-johnson-w261-hw5_3'])
with eda_mr.make_runner() as runner:
    runner.run()

In [24]:
!head out/count.txt
!head out/density.txt
!head out/length.txt
!aws s3 cp out/top_1000.txt s3://ucb-mids-mls-richjohnson

"[""COUNT"", ""the""]"	5490815394
"[""COUNT"", ""of""]"	3698583299
"[""COUNT"", ""to""]"	2227866570
"[""COUNT"", ""in""]"	1421312776
"[""COUNT"", ""a""]"	1361123022
"[""COUNT"", ""and""]"	1149577477
"[""COUNT"", ""that""]"	802921147
"[""COUNT"", ""is""]"	758328796
"[""COUNT"", ""be""]"	688707130
"[""COUNT"", ""as""]"	492170314
"[""DENSITY"", ""xxxx""]"	11.55729167
"[""DENSITY"", ""blah""]"	8.074159907
"[""DENSITY"", ""nnn""]"	7.533333333
"[""DENSITY"", ""na""]"	6.201749131
"[""DENSITY"", ""oooooooooooooooo""]"	4.921875
"[""DENSITY"", ""nd""]"	4.854305727
"[""DENSITY"", ""llll""]"	4.511627907
"[""DENSITY"", ""oooooo""]"	4.169650013
"[""DENSITY"", ""ooooo""]"	3.858637193
"[""DENSITY"", ""lillelu""]"	3.762452107
LENGTH_DISTRIBUTION	"[159, 2]"
LENGTH_DISTRIBUTION	"[128, 1]"
LENGTH_DISTRIBUTION	"[119, 2]"
LENGTH_DISTRIBUTION	"[106, 1]"
LENGTH_DISTRIBUTION	"[103, 1]"
LENGTH_DISTRIBUTION	"[91, 2]"
LENGTH_DISTRIBUTION	"[90, 1]"
LENGTH_DISTRIBUTION	"[89, 1]"
LENGTH_DISTRIBUTION	"[86, 1]"
LENGTH

###Results
The most frequent words were, as expected, stop words. Once we get beyond the first 100 or so they start to get more interesting.
Additionally, there were two (2) 5-grams with length 159 characters.
Density provided some confusing results. The fact that nonsense words dominate the top of the density chart is an unexpected phenomenon.

##HW 5.4
In this part of the assignment we will focus on developing methods
for detecting synonyms, using the Google 5-grams dataset. To accomplish
this you must script two main tasks using MRJob:

(1) Build stripes of word co-ocurrence for the top 10,000
most frequently appearing words across the entire set of 5-grams,
and output to a file in your bucket on s3 (bigram analysis, though the words are non-contiguous).

(2) Using two (symmetric) comparison methods of your choice 
(e.g., correlations, distances, similarities), pairwise compare 
all stripes (vectors), and output to a file in your bucket on s3.

==Design notes for (1)==
For this task you will be able to modify the pattern we used in HW 3.2
(feel free to use the solution as reference). To total the word counts 
across the 5-grams, output the support from the mappers using the total 
order inversion pattern:

<*word,count>

to ensure that the support arrives before the cooccurrences.

In addition to ensuring the determination of the total word counts,
the mapper must also output co-occurrence counts for the pairs of
words inside of each 5-gram. Treat these words as a basket,
as we have in HW 3, but count all stripes or pairs in both orders,
i.e., count both orderings: (word1,word2), and (word2,word1), to preserve
symmetry in our output for (2).

==Design notes for (2)==
For this task you will have to determine a method of comparison.
Here are a few that you might consider:

- Spearman correlation
- Euclidean distance
- Taxicab (Manhattan) distance
- Shortest path graph distance (a graph, because our data is symmetric!)
- Pearson correlation
- Cosine similarity
- Kendall correlation
...

However, be cautioned that some comparison methods are more difficult to
parallelize than others, and do not perform more associations than is necessary, 
since your choice of association will be symmetric.



###Find the top 1000 words and upload to S3

In [1]:
# Re-do as head -n 1000 | perl -c r/^.*? (/ 

word_counts = 'datafiles/count.txt'
top_10000_words = 'datafiles/top_10000.txt'

with open(word_counts, 'r') as fh:
    i = 0
    with open(top_10000_words, 'w') as rf:
        for line in fh.readlines():
            line = line.strip().split('\t')
            line[0] = line[0].strip('[]"')
            line[0] = line[0].split(',')[1].strip(' "') #Just keep the word and the count
            line[1] = int(line[1])
            rf.write(line[0]+'\n')
            i += 1
            if i == 10000:
                break


In [9]:
!head top_10000.txt
!aws s3 cp datafiles/top_10000.txt s3://ucb-mids-mls-richjohnson

the
of
to
in
a
and
that
is
be
as


###Create Stripes

In [3]:
%%writefile hw_5_4_mrjob.py
#!/usr/bin/python

from mrjob.job import MRJob
import urllib2

class Stripes(MRJob):
    def mapper_init(self):
        '''
        Load the top 10,000 words from an S3 bucket
        '''
        top_10000 = 'https://s3.amazonaws.com/ucb-mids-mls-richjohnson/top_10000.txt'
        a = urllib2.urlopen(top_10000)
        self.top_words = a.read().strip().split('\n')
        self.stripe = {}
        
    def mapper(self, _, line):
        line = line.strip().split('\t')
        ngram = line[0].lower().strip().split()
        
        # word1 and word2 must both be in top_words
        for word1 in ngram:
            if word1 in self.top_words:
                if word1 not in self.stripe.keys():
                    self.stripe[word1] = {}                
                for word2 in ngram:
                    if word2 in self.top_words and word1 != word2:
                        if word2 not in self.stripe[word1].keys():
                            self.stripe[word1][word2] = 0
                        self.stripe[word1][word2] += int(line[1])
    
    def mapper_final(self):
        for key1, subdict in self.stripe.iteritems():
            yield key1, subdict

    def reducer(self, key, values):
        stripe = {}
        for v in values:
            for k in v.keys():
                if k not in stripe.keys():
                    stripe[k] = 0
                stripe[k] += v[k]

        s = sum(stripe.values())
        for k, v in stripe.iteritems():
            stripe[k] = 1.0 * v / s
        yield key, stripe

if __name__ == '__main__':
    Stripes.run()

Overwriting hw_5_4_mrjob.py


In [1]:
%load_ext autoreload
%autoreload 2
#Driver
from hw_5_4_mrjob import Stripes

stripes_mr = Stripes(args=['s3://filtered-5grams',  '-r', 'emr', 
                           '--conf-path=mrjob.conf', '--no-output', '--output-dir=s3://ucb-mids-mls-richjohnson/out5-4-stripes'])
with stripes_mr.make_runner() as runner:
    runner.run()

###Find Synonyms using Euclidean Distance

After my conversation with Jake, he suggested a new way to attack this problem that I had not thought of before. Unfortunately, I did not have the time to implement it before the submission deadline, however given the time, I would do the following:
- I would restructure my mapper to emit each value for each word in the stripe in the form (column #, word), value. This would send all values in a column to the same mapper which would allow me to calculate every 10k choose 2 squared difference in values (to calculate Euclidean distance). Then, I would use another step to bring all the values back together and have a much simplter sum of squared differences for each 10k choose 2 words.

In [7]:
%%writefile hw_5_4_euc.py
#!/usr/bin/python

from mrjob.job import MRJob
from mrjob.step import MRStep
import ast
import urllib2

class EuclideanSim(MRJob):
    def steps(self):
        return[
            MRStep(mapper_init=self.mapper_init,
                   mapper=self.mapper,
                   reducer_init=self.reducer_init,
                   reducer=self.reducer,
                   reducer_final=self.reducer_final),
            MRStep(reducer=self.combine_squared_errors),
            MRStep(reducer=self.find_minimum)
        ]
    
    def mapper_init(self):
        '''
        Load the top 10,000 words from an S3 bucket
        '''
        top_10000 = 'https://s3.amazonaws.com/ucb-mids-mls-richjohnson/top_10000.txt'
        a = urllib2.urlopen(top_10000)
        self.top_words = a.read().strip().split('\n')
        
    def mapper(self, _, line):
        line = line.strip().split('\t')
        word = line[0]
        features = ast.literal_eval(line[1])
        
        # each word in top_words acts as a "column" in the 10k X 10k matrix
        for column in self.top_words:
            try:
                yield column, (word, features[column])
            except KeyError:
                # Yielding 0 increases overhead, but simplifies the algorithm
                # Re-write to decrease overhead if there is time
                yield column, (word, 0)
            
    def reducer_init(self):
        self.columns = {}
        
    def reducer(self, key, values):
        ''' 
        Each reduce key should have exactly 10,000 associated values, though many may be 0.
        This should have no problem fitting in memory.
        '''
        for v in values:
            self.columns[v[0]] = v[1]
        
    def reducer_final(self):
        for key1, val1 in self.columns.iteritems():
            for key2, val2 in self.columns.iteritems():
                # Only do in lexographic order and ensure keys aren't equal
                if key1 < key2:
                    yield (key1, key2), (val1-val2)**2
                    
    def combine_squared_errors(self, key, values):
        # Sqrt is not necessary; find_minimum() cares about the ordering not the actual Euc dist
        yield key[0], (key[1], sum(values)) 
        
    def find_minimum(self, key, values):
        minimum = 99999999
        min_key = None
        for k2, v in values:
            if v < minimum and v > 0.0: # If v == 0, something went wrong:
                minimum = v
                min_key = k2
        yield key, (min_key, minimum)

if __name__ == '__main__':
    EuclideanSim.run()

Overwriting hw_5_4_euc.py


In [9]:
%load_ext autoreload
%autoreload 2
from hw_5_4_euc import EuclideanSim

cos_mr = EuclideanSim(args=['-r', 'emr', 's3://ucb-mids-mls-richjohnson/out5-4-stripes/', 
                            '--conf-path=mrjob.conf', '--no-output', 
                            '--output-dir=s3://ucb-mids-mls-richjohnson/5-4-euc'])
with cos_mr.make_runner() as runner:
    runner.run()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


###Find Synonyms using Cosine Similarity

In order to calculate cosine similarity, I first need to determine the magnitude of each vector. I elected to do this in a separate MR job.

In [24]:
%%writefile hw_5_4_mags.py
from mrjob.job import MRJob
import ast

class NormVecs(MRJob):
    def mapper(self, _, line):
        line = line.split('\t')
        word = line[0]
        d = ast.literal_eval(line[1])
        mag = sum([x**2 for x in d.values()]) ** 0.5
        yield word, mag
        
    def reducer(self, key, values):
        for v in values:
            yield key, v
        
if __name__ == '__main__':
    NormVecs.run()

Overwriting hw_5_4_mags.py


Cosine similarity is calculated in a similar manner (algorithmically) as the Euclidean distance above. vectors are split on columns, with data being sent to mappers keyed on which column it came from.

In [34]:
%%writefile hw_5_4_cos.py
#!/usr/bin/python

from mrjob.job import MRJob
from mrjob.step import MRStep
import ast
import urllib2

class CosineSim(MRJob):
    def steps(self):
        return[
            MRStep(mapper_init=self.mapper_init,
                   mapper=self.mapper,
                   reducer_init=self.reducer_init,
                   reducer=self.reducer,
                   reducer_final=self.reducer_final),
            MRStep(reducer_init=self.get_mags,
                   reducer=self.calculate_cosine_sim),
            MRStep(reducer=self.find_minimum)
        ]
    
    def mapper_init(self):
        '''
        Load the top 10,000 words from an S3 bucket
        '''
        top_10000 = 'https://s3.amazonaws.com/ucb-mids-mls-richjohnson/top_10000.txt'
        a = urllib2.urlopen(top_10000)
        self.top_words = a.read().strip().split('\n')
        
    def mapper(self, _, line):
        line = line.strip().split('\t')
        word = line[0]
        features = ast.literal_eval(line[1])
        
        # each word in top_words acts as a "column" in the 10k X 10k matrix
        for column in self.top_words:
            try:
                yield column, (word, features[column])
            except KeyError:
                # Yielding 0 increases overhead, but simplifies the algorithm
                # Re-write to decrease overhead if there is time
                yield column, (word, 0)
            
    def reducer_init(self):
        self.columns = {}
        
    def reducer(self, key, values):
        ''' 
        Each reduce key should have exactly 10,000 associated values, though many may be 0.
        This should have no problem fitting in memory.
        '''
        for v in values:
            self.columns[v[0]] = v[1]
        
    def reducer_final(self):
        for key1, val1 in self.columns.iteritems():
            for key2, val2 in self.columns.iteritems():
                # Only do in lexographic order and ensure keys aren't equal
                if key1 < key2:
                    yield (key1, key2), (val1*val2)
    
    def get_mags(self):
        self.magnitudes = {}        
        link = 'https://s3.amazonaws.com/ucb-mids-mls-richjohnson/5-4-magnitudes/part-00000'
        a = urllib2.urlopen(link)
        lines = a.read().strip().split('\n')
        for line in lines:
            line = line.split('\t')
            self.magnitudes[line[0].strip('"\\')] = float(line[1])
    
    def calculate_cosine_sim(self, key, values):    
        
        key0 = key[0].strip('"\\')
        key1 = key[1].strip('"\\')
        yield key0, (key1, 1.0 * sum(values) / (self.magnitudes[key0] * self.magnitudes[key1])) 
    
    def find_minimum(self, key, values):
        minimum = 99999999
        min_key = None
        for k2, v in values:
            v = float(v)
            if v < minimum and v > 0.0: # If v == 0, something went wrong:
                minimum = v
                min_key = k2
        yield key.strip('"\\'), (min_key.strip('"\\', minimum)

if __name__ == '__main__':
    CosineSim.run()

Overwriting hw_5_4_cos.py


In [25]:
%load_ext autoreload
%autoreload 2
from hw_5_4_mags import NormVecs

mag_mr = NormVecs(args=['-r', 'emr', 's3://ucb-mids-mls-richjohnson/out5-4-stripes/', '--conf-path=mrjob.conf',
                        '--no-output', '--output-dir=s3://ucb-mids-mls-richjohnson/5-4-magnitudes'])

with mag_mr.make_runner() as runner:
    runner.run()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
%load_ext autoreload
%autoreload 2
from hw_5_4_cos import CosineSim

cos_mr = CosineSim(args=['-r', 'emr', 's3://ucb-mids-mls-richjohnson/out5-4-1/', '--conf-path=mrjob.conf',
                         '--no-output', '--output-dir=s3://ucb-mids-mls-richjohnson/5-4-cos'])
with cos_mr.make_runner() as runner:
    runner.run()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##HW 5.5
In this part of the assignment you will evaluate the success of you synonym detector.
Take the top 1,000 closest/most similar/correlative pairs of words as determined
by your measure in (2), and use the synonyms function in the accompanying
python code:

nltk_synonyms.py

Note: This will require installing the python nltk package:

http://www.nltk.org/install.html

and downloading its data with nltk.download().

For each (word1,word2) pair, check to see if word1 is in the list, 
synonyms(word2), and vice-versa. If one of the two is a synonym of the other, 
then consider this pair a 'hit', and then report the precision, recall, and F1 measure  of 
your detector across your 1,000 best guesses. Report the macro averages of these measures.

In [42]:
#!/usr/bin/python
import nltk
from nltk.corpus import wordnet as wn
from itertools import combinations
import sys, re
import ast

TOP_10K = './output/frequent_unigrams_10K.txt'
EUCLID = './output/euc.txt'
COSINE = './output/cos.txt'

def synonyms(string):
    syndict = {}
    for i,j in enumerate(wn.synsets(string)):
        syns = j.lemma_names()
        for syn in syns:
            syndict.setdefault(syn,1)
    return syndict.keys()

def synonym_pairs():
    # create pairs with the full collection of unique 
    # synonym (word1,word2) pairs subject to top 10K words
    wordList = []
    for line in open(TOP_10K).read().strip().split('\n'):
        word = line.split('\t')[0].strip(' "')
        wordList.append(word)

    pairs = {}
    for word in wordList:
        syns = synonyms(word)
        keepSyns = []
        for syn in syns:
            syn = str(re.sub("_"," ",syn))
            if syn != word:
                if syn in wordList:
                    keepSyns.append(syn)
        for syn in keepSyns:
            pair = ",".join(sorted([word,syn]))
            pairs[pair] = 1

    return pairs

def calculate_measures(similar_words, threshold):
    rownum = 0
    hits = []
    
    #synonyms = synonym_pairs()

    for line in open(similar_words).readlines():
        line = line.strip().split("\t")
        line[0] = line[0].strip('"\\"')
        line[1] = line[1].split(',')
        line[1][0] = line[1][0].strip('"\\[]')
        line[1][1] = line[1][1].strip('"\\[]')
        # pair, distance        
        p = (line[0], line[1][0])
        d = float(line[1][1])
        
        if p[0] in synonyms(p[1]) or p[1] in synonyms(p[0]):
            hits.append(1)
        else:
            hits.append(0)

        rownum += 1       
    # assume words below threshold are hits
    predictions = [1]*threshold + [0]*(rownum-threshold)

    # determine measures
    tp = 0
    fn = 0
    fp = 0
    tn = 0

    for i in range(len(predictions)):
        # true positives
        if hits[i] == 1 and predictions[i] == 1:
            tp += 1
        # true negatives
        elif hits[i] == 0 and predictions[i] == 0:
            tn += 1
        # false negatives
        elif hits[i] == 1 and predictions[i] == 0:
            fn += 1
        # false positives
        else:
            fp += 1

    accuracy = float(tp + tn) / len(predictions)
    recall = float(tp) / float(tp + fn)
    precision = float(tp) / float(tp + fp)

    print "Accuracy: {}".format(accuracy)
    print "Precision: {}".format(precision)
    print "Recall: {}".format(recall)
    
    try:
        print "F1 Score: {}".format(2 * (precision*recall) / (precision + recall))
    except ZeroDivisionError:
        print "F1 Score: Inf"

In [16]:
!mkdir output
!aws s3 cp s3://ucb-mids-mls-richjohnson/top_10000.txt ./output/frequent_unigrams_10K.txt
!aws s3 cp s3://ucb-mids-mls-richjohnson/5-4-euc ./output/euc/ --recursive
!aws s3 cp s3://ucb-mids-mls-richjohnson/5-4-cos ./output/cos/ --recursive
!cat output/euc/part* >> output/euc.txt
!cat output/cos/part* >> output/cos.txt

download: s3://ucb-mids-mls-richjohnson/top_10000.txt to output/frequent_unigrams_10K.txt
download: s3://ucb-mids-mls-richjohnson/5-4-euc/_SUCCESS to output/euc/_SUCCESS
download: s3://ucb-mids-mls-richjohnson/5-4-euc/part-00009 to output/euc/part-00009
download: s3://ucb-mids-mls-richjohnson/5-4-euc/part-00010 to output/euc/part-00010
download: s3://ucb-mids-mls-richjohnson/5-4-euc/part-00002 to output/euc/part-00002
download: s3://ucb-mids-mls-richjohnson/5-4-euc/part-00008 to output/euc/part-00008
download: s3://ucb-mids-mls-richjohnson/5-4-euc/part-00011 to output/euc/part-00011
download: s3://ucb-mids-mls-richjohnson/5-4-euc/part-00007 to output/euc/part-00007
download: s3://ucb-mids-mls-richjohnson/5-4-euc/part-00012 to output/euc/part-00012
download: s3://ucb-mids-mls-richjohnson/5-4-euc/part-00014 to output/euc/part-00014
download: s3://ucb-mids-mls-richjohnson/5-4-euc/part-00013 to output/euc/part-00013
download: s3://ucb-mids-mls-richjohnson/5-4-euc/part-00001 to output/euc/p

In [43]:
threshold = 1000
# euclidean
calculate_measures(EUCLID, threshold)

# cosine similarity
calculate_measures(COSINE, threshold)

Accuracy: 0.897889788979
Precision: 0.002
Recall: 0.08
F1 Score: 0.00390243902439
Accuracy: 0.798266827892
Precision: 0.0
Recall: 0.0
F1 Score: Inf


##HW 5.6 (optional)
There are many good ways to build our synonym detectors, so for optional homework, 
measure co-occurrence by (left/right/all) consecutive words only, 
or make stripes according to word co-occurrences with the accompanying 
2-, 3-, or 4-grams (note here that your output will no longer 
be interpretable as a network) inside of the 5-grams.


##Hw 5.7 (optional)
Once again, benchmark your top 1,000 associations (as in 5.5), this time for your
results from 5.6. Has your detector improved?